In [6]:
import warnings
warnings.filterwarnings('ignore')

import geopandas
import r5py
import shapely
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime
import copy

import os
import snman
from snman import osmnx_customized as oxc
from snman.constants import *

PERIMETER = '_accessibility_debug'

# Set these paths according to your own setup
data_directory = os.path.join('C:',os.sep,'Users','lballo','polybox','Research','SNMan','SNMan Shared','data_v2')
inputs_path = os.path.join(data_directory, 'inputs')
process_path = os.path.join(data_directory, 'process', PERIMETER)
outputs_path = os.path.join(data_directory, 'outputs', PERIMETER)

CRS_internal = 2056      # for Zurich

In [7]:
REBUILD_INPUTS = True
STATE = 'before'
N_ORIGIN_CELLS = 2500
SAMPLE_RANGE = [0, 10]

In [8]:
# read transport network and timetable for r5
if 1:
    r5_transit_network = r5py.TransportNetwork(
        os.path.join(outputs_path, 'before_oneway_links_default.osm.pbf'),
        #os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'vbz_2024.zip'),
        #os.path.join(inputs_path, 'switzerland', 'switzerland', 'gtfs', 'gtfs_fp2024_2024-06-27_mod.zip'),
    )

In [9]:
if REBUILD_INPUTS:
    G_modes = {}
    L_modes = {}

In [10]:
if REBUILD_INPUTS:
    # Load small street graph
    G = snman.io.load_street_graph(
        os.path.join(data_directory, outputs_path, 'G_edges.gpkg'),
        os.path.join(data_directory, outputs_path, 'G_nodes.gpkg'),
        crs=CRS_internal
    )
    
    if STATE == 'before':
        ln_desc = KEY_LANES_DESCRIPTION
    else:
        ln_desc = KEY_LANES_DESCRIPTION_AFTER
    
    # create mode graphs for other modes
    for mode in [MODE_CYCLING, MODE_FOOT]:
        print(f'Make street and lane graphs for {mode}')
        G_modes[mode] = copy.deepcopy(G)
        snman.street_graph.filter_lanes_by_modes(
            G_modes[mode], [mode], lane_description_key=ln_desc
        )
        L_modes[mode] = snman.lane_graph.create_lane_graph(
            G_modes[mode], lanes_attribute=ln_desc
        )


Make street and lane graphs for cycling
Make street and lane graphs for foot


In [ ]:
if REBUILD_INPUTS:
    # load the new car street graph created from MATSim output
    G_modes[MODE_PRIVATE_CARS] = snman.io.load_street_graph(
        os.path.join(outputs_path, f'tt_{STATE}_edges.gpkg'),
        os.path.join(outputs_path, f'tt_{STATE}_nodes.gpkg'),
        crs=CRS_internal,
        unstringify_attributes={'lanes': snman.space_allocation.space_allocation_from_string}
    )
    
    # we must avoid that the start/end point of trips are matched onto nodes that not accessible by car
    # therefore, we reduce the graph only to those links and nodes that are accessible by car
    snman.street_graph.filter_lanes_by_modes(
        G_modes[MODE_PRIVATE_CARS], [MODE_PRIVATE_CARS], lane_description_key='lanes'
    )
    
    L_modes[MODE_PRIVATE_CARS] = snman.lane_graph.create_lane_graph(
        G_modes[MODE_PRIVATE_CARS], lanes_attribute='lanes', cast_attributes={'matsim_tt_cars': 'car_18:00'}
    )
    

In [7]:
# Import Statent grid
if REBUILD_INPUTS:
    statent = gpd.read_parquet(
        os.path.join(inputs_path, 'switzerland', 'switzerland', 'statent', 'STATENT_2021_ebc_10_reduced_fields.gzip')
    )
    statent.rename(columns={'RELI': 'id'}, inplace=True)

In [8]:
if REBUILD_INPUTS:
    # read statpop that has been pre-joined with statent
    statpop_with_statent_ids = gpd.read_parquet(
        os.path.join(inputs_path, 'switzerland', 'switzerland', 'statpop', 'statpop2017_with_statent_reduced_columns.gzip')
    )

In [9]:
perimeter = snman.io.import_geofile_to_gdf(
    os.path.join(inputs_path, 'perimeters', 'perimeters.shp'), index='id'
).geometry['ebc_zrh_v01'].simplify(500, preserve_topology=True)
statent_in_perimeter = statent[statent.within(perimeter)]

In [20]:
cells = list(statent_in_perimeter['id'].sample(n=N_ORIGIN_CELLS, random_state=0))[SAMPLE_RANGE[0]:SAMPLE_RANGE[1]]

def miniwrapper(args):
    cell, i, N = args
    print(f'calculating cell {cell} ({i}/{N})')
    a = snman.accessibility.calculate_accessibility_for_statent_cell(
        r5_transit_network,
        G_modes,
        L_modes,
        statpop_with_statent_ids, statent, cell,
        datetime.datetime(2024, 2, 22, 18, 00),
        distance_limit=50*1000,
        population_sample=0.05,
        destinations_sample=0.1
    )
    return a

accessibility = pd.concat(
    map(
        miniwrapper,
        zip(
            cells,
            range(len(cells)),
            [len(cells)] * len(cells)
        )
    ),
    ignore_index=True
)


print('exporting file')
snman.io.export_gdf(
    accessibility,
    os.path.join(outputs_path, f'accessibility_{STATE}.gpkg')
)

calculating cell 70042281 (0/10)
calculating cell 68132486 (1/10)
calculating cell 68672447 (2/10)
calculating cell 69702513 (3/10)
calculating cell 67962611 (4/10)
calculating cell 66382604 (5/10)
calculating cell 68322416 (6/10)
calculating cell 66682596 (7/10)
calculating cell 68172475 (8/10)
calculating cell 69482396 (9/10)
exporting file


In [22]:
accessibility

,age,sex,maritalstatus,residencepermit,residentpermit,statent_id,accessibility,accessibility_cycling,accessibility_foot,accessibility_private_cars,accessibility_transit,geometry,record
0,38,1,1,301,3,68672447.0,449.042529,574.198628,227.824450,599.103377,420.602192,POINT (2686663.001 1244743.001),2
1,48,2,2,-2,-2,68672447.0,449.042045,574.198628,227.824450,599.103377,420.602192,POINT (2686744.001 1244666.001),2
2,56,1,2,201,2,68672447.0,449.041551,574.198628,227.824450,599.103377,420.602192,POINT (2686703.001 1244729.001),2
3,13,1,1,-2,-2,69702513.0,331.572043,404.511378,138.711689,503.801828,280.856796,POINT (2697032.001 1251290.001),3
4,20,2,1,-2,-2,69702513.0,331.571934,404.511378,138.711689,503.801828,280.856796,POINT (2696996.001 1251309.001),3
5,37,1,2,-2,-2,69702513.0,331.571047,404.511378,138.711689,503.801828,280.856796,POINT (2697040.001 1251272.001),3
6,53,1,2,-2,-2,69702513.0,331.570150,404.511378,138.711689,503.801828,280.856796,POINT (2696987.001 1251322.001),3
7,18,2,1,-2,-2,69702513.0,331.572043,404.511378,138.711689,503.801828,280.856796,POINT (2696970.001 1251256.001),3
8,52,2,2,302,3,67962611.0,366.608394,457.270747,156.450600,572.622581,292.256550,POINT (2679635.001 1261059.001),4
9,27,1,2,201,2,67962611.0,366.714442,457.270747,156.450600,572.622581,292.256550,POINT (2679618.001 1261094.001),4


In [11]:
if 0:
    accessibility_before = snman.io.import_geofile_to_gdf(
        os.path.join(outputs_path, 'accessibility_before.gpkg')
    )
    
    accessibility_after = snman.io.import_geofile_to_gdf(
        os.path.join(outputs_path, 'accessibility_after.gpkg')
    )

In [12]:
if 0:
    accessibility_keys = [
        'accessibility', 'accessibility_private_cars',
        'accessibility_cycling', 'accessibility_foot', 'accessibility_transit'
    ]
    
    accessibility_before_after = pd.merge(
        accessibility_before,
        accessibility_after[accessibility_keys],
        left_index=True, right_index=True,
        suffixes=['_before', '_after']
    )
    
    for key in accessibility_keys:
    
        accessibility_before_after[f'{key}_after-before'] = (
                accessibility_before_after[f'{key}_after']
                - accessibility_before_after[f'{key}_before']
        )
    
    snman.io.export_gdf(
        accessibility_before_after,
        os.path.join(outputs_path, 'accessibility_before_after.gpkg')
    )